<a href="https://colab.research.google.com/github/SohailVibeCoder/Olist-ML-project/blob/main/Programming%20Assignment%20-%20Final%20Final%20Model%20i%20promise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data cleaning & Feature Engineering

In [11]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import os

# --- 1. SETUP and INITIAL DATA LOADING ---
drive.mount('/content/drive')
# Assuming your Olist files are in your Drive root folder
os.chdir("/content/drive/MyDrive/")

# Load all required original datasets
orders_df = pd.read_csv("olist_orders_dataset.csv")
order_reviews_df = pd.read_csv("olist_order_reviews_dataset.csv")
order_payments_df = pd.read_csv("olist_order_payments_dataset.csv")
customers_df = pd.read_csv("olist_customers_dataset.csv")
order_items_df = pd.read_csv("olist_order_items_dataset.csv")
products_df = pd.read_csv("olist_products_dataset.csv")
sellers_df = pd.read_csv("olist_sellers_dataset.csv")
translation_df = pd.read_csv("product_category_name_translation.csv")
geolocation_df = pd.read_csv("olist_geolocation_dataset.csv") # <-- Geo Data


# --- 2. PERFORM INITIAL MERGE and CLEANING ---
full_df = (
    orders_df
    .merge(order_reviews_df, on='order_id', how='left')
    .merge(order_payments_df, on='order_id', how='left')
    .merge(customers_df, on='customer_id', how='left')
    .merge(order_items_df, on='order_id', how='left')
    .merge(products_df, on='product_id', how='left')
    .merge(sellers_df, on='seller_id', how='left')
    .merge(translation_df, on='product_category_name', how='left')
)

full_df.to_csv("RawMergedData.csv", index=False)


# Filter Delivered, Convert Dates, Drop Core Nulls
full_df = full_df[full_df['order_status'] == 'delivered'].copy()
date_cols = [
    'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
    'order_delivered_customer_date', 'order_estimated_delivery_date', 'shipping_limit_date',
    'review_creation_date', 'review_answer_timestamp'
]
for col in date_cols:
    full_df[col] = pd.to_datetime(full_df[col], errors='coerce')
full_df.dropna(subset=['price', 'product_id', 'review_score', 'product_category_name_english'], inplace=True)
with pd.option_context('mode.chained_assignment', None):
    full_df['review_comment_title'].fillna('', inplace=True)
    full_df['review_comment_message'].fillna('', inplace=True)

# --- 3. GEOLOCATION AGGREGATION AND MERGE ---
# Get median lat/lng for each zip code prefix
geo_median = geolocation_df.groupby('geolocation_zip_code_prefix').agg(
    geolocation_lat=('geolocation_lat', 'median'),
    geolocation_lng=('geolocation_lng', 'median')
).reset_index()

# Merge customer lat/lng
geo_customer = geo_median.rename(columns={'geolocation_zip_code_prefix': 'customer_zip_code_prefix', 'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_lng'})
full_df = full_df.merge(geo_customer, on='customer_zip_code_prefix', how='left')

# Merge seller lat/lng
geo_seller = geo_median.rename(columns={'geolocation_zip_code_prefix': 'seller_zip_code_prefix', 'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'})
full_df = full_df.merge(geo_seller, on='seller_zip_code_prefix', how='left')

# Drop rows missing necessary geo-coordinates
full_df.dropna(subset=['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng'], inplace=True)
df = full_df.copy()

# =================================================================
# ADVANCED FEATURE ENGINEERING (New Geo + Old RFM/Product Logic)
# =================================================================

# --- 4. Haversine Distance & K-Means Clustering ---

# Haversine Distance Function
def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

df['distance_km'] = haversine(
    df['customer_lat'], df['customer_lng'],
    df['seller_lat'], df['seller_lng']
)

# K-Means Clustering (K=10)
geo_coords = df[['customer_lat', 'customer_lng']].values
scaler = StandardScaler()
geo_coords_scaled = scaler.fit_transform(geo_coords)
K = 10
kmeans = KMeans(n_clusters=K, random_state=42, n_init='auto', max_iter=300, verbose=0)
df['customer_geo_cluster'] = kmeans.fit_predict(geo_coords_scaled)
df['customer_geo_cluster'] = df['customer_geo_cluster'].astype(object)


# --- 5. Create RFM, Delivery, and Volume Features (as in your original code) ---

# A. RFM Features
snapshot_date = df['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
rfm_df = df.groupby('customer_unique_id').agg(
    Recency=('order_purchase_timestamp', lambda x: (snapshot_date - x.max()).days),
    Frequency=('order_id', 'nunique'),
    Monetary=('payment_value', 'sum')
).reset_index()
df = df.merge(rfm_df, on='customer_unique_id', how='left')

# B. Delivery Metrics
df['delivery_time_delta'] = df['order_delivered_customer_date'] - df['order_purchase_timestamp']
df['actual_delivery_days'] = df['delivery_time_delta'].dt.days.fillna(df['delivery_time_delta'].dt.days.median())
df['delivery_performance_days'] = (
    df['order_estimated_delivery_date'] - df['order_delivered_customer_date']
).dt.days.fillna(0)

# C. Product Volume
df['product_volume_cm3'] = (df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm'])
volume_median = df['product_volume_cm3'].median()
with pd.option_context('mode.chained_assignment', None):
    df['product_volume_cm3'].fillna(volume_median, inplace=True)


# --- 6. One-Hot Encoding and Product Category Aggregation ---

# Categorical columns now include the new customer_geo_cluster
categorical_cols = ['product_category_name_english', 'payment_type', 'customer_geo_cluster']
df_encoded = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols, drop_first=True)

# Product Category Aggregation (as in your original code)
columns_to_drop_after_agg = []
product_prefix = 'product_category_name_english_'
category_map = {
    'Home_Kitchen_Comfort': ['air_conditioning', 'home_appliances', 'home_appliances_2', 'home_comfort_2', 'home_confort', 'housewares', 'kitchen_dining_laundry_garden_furniture', 'la_cuisine', 'small_appliances', 'small_appliances_home_oven_and_coffee'],
    'Fashion_Apparel': ['fashio_female_clothing', 'fashion_male_clothing', 'fashion_childrens_clothes', 'fashion_underwear_beach', 'fashion_shoes', 'fashion_sport', 'fashion_bags_accessories', 'luggage_accessories', 'watches_gifts'],
    'Electronics_Technology': ['audio', 'cine_photo', 'computers', 'computers_accessories', 'consoles_games', 'electronics', 'fixed_telephony', 'music', 'musical_instruments', 'tablets_printing_image', 'telephony'],
    'Tools_Construction': ['construction_tools_construction', 'construction_tools_lights', 'construction_tools_safety', 'costruction_tools_garden', 'costruction_tools_tools', 'garden_tools', 'home_construction'],
    'Furniture_Decor': ['bed_bath_table', 'furniture_bedroom', 'furniture_decor', 'furniture_living_room', 'furniture_mattress_and_upholstery', 'office_furniture'],
    'Books_Media_Toys': ['books_general_interest', 'books_imported', 'books_technical', 'cds_dvds_musicals', 'dvds_blu_ray', 'toys', 'cool_stuff'],
    'Health_Personal_Care': ['health_beauty', 'perfumery', 'baby', 'diapers_and_hygiene'],
    'Art_Hobbies': ['art', 'arts_and_craftmanship', 'stationery', 'party_supplies', 'christmas_supplies'],
    'Food_Drink': ['drinks', 'food', 'food_drink'],
    'Automotive': ['auto'],
    'Other_Services': ['flowers', 'industry_commerce_and_business', 'market_place', 'pet_shop', 'security_and_services', 'signaling_and_security']
}
for new_category, detailed_categories in category_map.items():
    original_cols = [product_prefix + cat for cat in detailed_categories]
    existing_cols = [col for col in original_cols if col in df_encoded.columns]
    if existing_cols:
        df_encoded[new_category] = df_encoded[existing_cols].any(axis=1).astype(int)
        columns_to_drop_after_agg.extend(existing_cols)
sports_leisure_col = product_prefix + 'sports_leisure'
if sports_leisure_col in df_encoded.columns:
    df_encoded.rename(columns={sports_leisure_col: 'Sports_Leisure'}, inplace=True)
df_encoded = df_encoded.drop(columns=columns_to_drop_after_agg, errors='ignore')


# --- 7. Final Drop of Irrelevant or Redundant Columns ---
# IMPORTANT: This list ensures all raw IDs and the old state/region data are dropped.
columns_to_drop_final = [
    'order_id', 'customer_id', 'review_id', 'product_id', 'seller_id', 'customer_unique_id',
    'order_status', 'review_comment_title', 'review_comment_message', 'customer_city', 'seller_city',
    'product_category_name', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
    'order_delivered_customer_date', 'order_estimated_delivery_date', 'review_creation_date',
    'review_answer_timestamp', 'shipping_limit_date', 'product_length_cm', 'product_height_cm',
    'product_width_cm', 'customer_zip_code_prefix', 'seller_zip_code_prefix', 'delivery_time_delta',
    'order_item_id', 'product_category_name_english', 'payment_sequential', # <-- Ensure payment_sequential is dropped
    # DROPPING RAW GEO COORDINATES (now replaced by distance_km and clusters)
    'customer_lat', 'customer_lng', 'seller_lat', 'seller_lng',
    # DROPPING the old state columns that were used to create regions (Replaced by K-Means Clusters)
    'customer_state', 'seller_state'
]

final_ml_df = df_encoded.drop(columns=columns_to_drop_final, errors='ignore')
final_ml_df.to_csv("ml_ready_feature_table_V2.csv", index=False)
print(f"\n✅ Feature engineering complete. Final dataset saved as 'ml_ready_feature_table_V2.csv'. Shape: {final_ml_df.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


XGBoost Classifier

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from xgboost import XGBClassifier



# =================================================================
# SECTION 1: DATA LOADING AND SETUP
# =================================================================
# Load the final, advanced feature dataset
df_ml = pd.read_csv("ml_ready_feature_table_V2.csv")
print("dataset loaded. Shape:", df_ml.shape)

# Target variable: 1 = Good (≥4), 0 = Bad (<4)
y_class = (df_ml['review_score'] >= 4).astype(int)

# Drop review_score (target)
X = df_ml.drop(columns=['review_score'])

# --- 2. Train-Test Split and Class Weighting ---
X_train, X_test, y_train_class, y_test_class = train_test_split(
    X, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Prepare evaluation set
eval_set = [
    (X_train, y_train_class),
    (X_test, y_test_class)
]

# Calculate imbalance ratio for scale_pos_weight
count_class_1 = y_train_class.value_counts()[1]
count_class_0 = y_train_class.value_counts()[0]
scale_pos_weight = count_class_1 / count_class_0
print(f"Calculated scale_pos_weight: {scale_pos_weight:.2f}")

# =================================================================
# SECTION 2: MODEL TRAINING
# =================================================================
# --- 3. Train XGBoost Classifier (Using the standard hyper-parameters) ---
xgb_final_model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1,
    scale_pos_weight=scale_pos_weight,
    eval_metric="logloss"
)
print("\nTraining XGBoost Model...")
xgb_final_model.fit(
    X_train,
    y_train_class,
    eval_set=eval_set,
    verbose=False
)
print("Model training complete.")

# =================================================================
# SECTION 3: EVALUATION AND FEATURE IMPORTANCE
# =================================================================
# --- 5. Threshold Optimization (Focusing on the minority class: Bad Review) ---
y_proba = xgb_final_model.predict_proba(X_test)
y_proba_minority = y_proba[:, 0]  # Probability of being class 0 (bad review)

# Find the optimal threshold for the best F1 score on the Bad Review class
thresholds = np.linspace(0.50, 0.10, 20)
best_f1, best_threshold = 0, 0

for threshold in thresholds:
    y_pred_temp = np.where(y_proba_minority > threshold, 0, 1)
    # Calculate F1 for the minority class (pos_label=0)
    f1 = f1_score(y_test_class, y_pred_temp, pos_label=0)
    if f1 > best_f1:
        best_f1, best_threshold = f1, threshold

print(f"\nOptimal threshold: {best_threshold:.3f} (F1 for bad class = {best_f1:.3f})")


# --- 6. Final Evaluation (TEST Set + Confusion Matrix) ---
from sklearn.metrics import confusion_matrix

y_pred_final = np.where(y_proba_minority > best_threshold, 0, 1)

print("\n---  MODEL TEST SET PERFORMANCE ---")
print(classification_report(y_test_class, y_pred_final, target_names=['0 (Bad)', '1 (Good)']))

# Confusion Matrix
cm = confusion_matrix(y_test_class, y_pred_final)

print("\nConfusion Matrix (Test Set):")
print(cm)

# Pretty-print version
cm_df = pd.DataFrame(
    cm,
    index=['Actual Bad (0)', 'Actual Good (1)'],
    columns=['Predicted Bad (0)', 'Predicted Good (1)']
)

print("\nConfusion Matrix (Formatted):")
print(cm_df)



dataset loaded. Shape: (112657, 40)
Calculated scale_pos_weight: 3.31

Training XGBoost Model...
Model training complete.

Optimal threshold: 0.100 (F1 for bad class = 0.517)

---  MODEL TEST SET PERFORMANCE ---
              precision    recall  f1-score   support

     0 (Bad)       0.50      0.54      0.52      5223
    1 (Good)       0.86      0.84      0.85     17309

    accuracy                           0.77     22532
   macro avg       0.68      0.69      0.68     22532
weighted avg       0.77      0.77      0.77     22532


Confusion Matrix (Test Set):
[[ 2799  2424]
 [ 2801 14508]]

Confusion Matrix (Formatted):
                 Predicted Bad (0)  Predicted Good (1)
Actual Bad (0)                2799                2424
Actual Good (1)               2801               14508


Pearsons Coefficient Barchart (Magnitude adjusted)



In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. Load the advanced ML-ready dataset ---
df_original = pd.read_csv("ml_ready_feature_table_GEO_V2.csv")

# --- 2. Calculate Model-Agnostic Feature Importance (Absolute Correlation) ---
# Create the binary target variable (1 for good review, 0 for bad review)
df_original['review_score_binary'] = (df_original['review_score'] >= 4).astype(int)
TARGET_COLUMN = 'review_score_binary'

# Drop only the original review score column, as all other irrelevant columns
# (like payment_sequential) were already dropped in the Feature Engineering phase.
columns_to_exclude = ['review_score']
df_corr = df_original.drop(columns=columns_to_exclude, errors='ignore')

# Clean and ensure all columns are numeric
for col in df_corr.columns:
    if df_corr[col].dtype == 'bool':
        df_corr[col] = df_corr[col].astype(int)
df_corr.fillna(0, inplace=True)

# Calculate absolute correlation with the binary target
correlation_matrix = df_corr.corr()
target_correlation = correlation_matrix[TARGET_COLUMN].drop(TARGET_COLUMN)
absolute_correlation = target_correlation.abs()

# Select the top 10 most correlated features
top_features_corr = absolute_correlation.sort_values(ascending=False).head(10)

print("\nTop 10 Features based on Absolute Correlation with Binary Review Score (Geo V2):")
print(top_features_corr)


# --- 3. Generate the Bar Chart ---
plt.figure(figsize=(10, 6))
top_features_corr.sort_values(ascending=True).plot(kind='barh', color='darkgreen')
plt.title(
    'Top 10 Feature Strengths: Absolute Pearson Correlation (Advanced Geo V2)',
    fontsize=14
)
plt.xlabel("Absolute Correlation Coefficient ($|r|$)", fontsize=12)
plt.ylabel("Feature Name", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('top_features_correlation_geo_v2.png')
plt.close()

print("\nBar chart saved as 'top_features_correlation_geo_v2.png'.")


Top 10 Features based on Absolute Correlation with Binary Review Score (Geo V2):
actual_delivery_days         0.268820
delivery_performance_days    0.200433
Monetary                     0.083553
payment_value                0.067759
Furniture_Decor              0.065275
customer_geo_cluster_1       0.047985
distance_km                  0.047473
payment_installments         0.042257
freight_value                0.028926
Books_Media_Toys             0.028753
Name: review_score_binary, dtype: float64

Bar chart saved as 'top_features_correlation_geo_v2.png'.


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Load the dataset ---
df_original = pd.read_csv("ml_ready_feature_table_GEO_V2.csv")

# --- 2. Create binary satisfaction target ---
df_original['review_score_binary'] = (df_original['review_score'] >= 4).astype(int)
TARGET_COLUMN = 'review_score_binary'

# Remove raw score
df_corr = df_original.drop(columns=['review_score'], errors='ignore')

# Clean boolean columns
for col in df_corr.columns:
    if df_corr[col].dtype == 'bool':
        df_corr[col] = df_corr[col].astype(int)

df_corr.fillna(0, inplace=True)

# --- 3. Pearson correlations ---
corr_matrix = df_corr.corr()
target_corr = corr_matrix[TARGET_COLUMN].drop(TARGET_COLUMN)

# --- 4. Select only negative features (dissatisfaction drivers) ---
negative_corr = target_corr[target_corr < 0]

# Top 5 most negative correlations
top5_negative = negative_corr.sort_values().head(5)

# --- 5. Business-friendly naming ---
#name_map = {
   # 'delivery_performance_days': 'Delivery: Days Ahead/Behind Estimate',
  #  'actual_delivery_days': 'Delivery: Total Shipping Days',
   # 'distance_km': 'Logistics: Customer-Seller Distance (KM)',
   # 'Monetary': 'Customer Value (Monetary)',
   # 'payment_value': 'Total Payment Value',
   # 'Furniture_Decor': 'Product: Furniture/Decor',
   # 'customer_geo_cluster_1': 'Geo Hub 1',
   # 'payment_installments': 'Payment Installments',
   # 'freight_value': 'Item Freight Value',
   # 'Books_Media_Toys': 'Product: Books/Media/Toys'
#}

#top5_negative.index = top5_negative.index.map(lambda x: name_map.get(x, x))

# Convert to positive for plotting (but it's still negative behind the scenes)
top5_plot = top5_negative.abs().sort_values(ascending=False)

# --- 6. Plot: clean, blue gradient, no outlines, no numbers ---
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 5))

# Blue gradient
colors = sns.color_palette("Blues", n_colors=len(top5_plot))

plt.bar(
    x=np.arange(len(top5_plot)),
    height=top5_plot.values,
    color=colors
)

plt.xticks(
    ticks=np.arange(len(top5_plot)),
    labels=top5_plot.index,
    rotation=45,
    ha='right',
    fontsize=10
)

plt.title(
    "Top 5 Drivers of Customer Dissatisfaction",
    fontsize=16,
    fontweight="bold",
    pad=15
)

plt.ylabel("Pearsons Correlation Coefficient", fontsize=12)
plt.xlabel("Feature", fontsize=12)

plt.grid(axis='y', linestyle='--', alpha=0.4)
sns.despine(left=True, bottom=True)

plt.tight_layout()
plt.savefig("top_5_negative_pearson_.png", dpi=300)
plt.close()

print("\nSaved as 'top_5_negative_pearson.png'")



Saved as 'top_5_negative_pearson.png'
